### Data Preprocessing

In [34]:
import numpy as np
import pandas as pd
import pickle
from sklearn.metrics import accuracy_score
from sklearn import metrics

In [35]:
df = pd.read_csv('data/Telco-Customer-Churn.csv')
df['TotalCharges'] = df['TotalCharges'].replace(" ", 0).astype('float32')

In [36]:
cat_features = df.drop(['customerID','TotalCharges', 'MonthlyCharges', 'SeniorCitizen', 'tenure', 'Churn'],axis=1).columns
cat_features

Index(['gender', 'Partner', 'Dependents', 'PhoneService', 'MultipleLines',
       'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection',
       'TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract',
       'PaperlessBilling', 'PaymentMethod'],
      dtype='object')

In [37]:
df[cat_features].head()

,gender,Partner,Dependents,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod
0,Female,Yes,No,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check
1,Male,No,No,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check
2,Male,No,No,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check
3,Male,No,No,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic)
4,Female,No,No,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check


In [38]:
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder(sparse=False)
ohe.fit(df[cat_features])

OneHotEncoder(sparse=False)

In [39]:
dff = ohe.transform(df[cat_features])
dff = pd.DataFrame(dff, columns=ohe.get_feature_names())
dff = pd.concat([dff, df[['SeniorCitizen', 'MonthlyCharges', 'TotalCharges', 'tenure']]], axis=1)

C:\python3.8\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [40]:
dff.head()

,x0_Female,x0_Male,x1_No,x1_Yes,x2_No,x2_Yes,x3_No,x3_Yes,x4_No,x4_No phone service,...,x13_No,x13_Yes,x14_Bank transfer (automatic),x14_Credit card (automatic),x14_Electronic check,x14_Mailed check,SeniorCitizen,MonthlyCharges,TotalCharges,tenure
0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0,29.85,29.850000,1
1,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0,56.95,1889.500000,34
2,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0,53.85,108.150002,2
3,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,...,1.0,0.0,1.0,0.0,0.0,0.0,0,42.30,1840.750000,45
4,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0,70.70,151.649994,2


In [41]:
bin_dict = {'No':0, 'Yes':1}
df.Churn = df.Churn.map(bin_dict)

### Modeling

In [42]:
from sklearn.model_selection import train_test_split

X = dff.values
y = df.Churn

X_train, X_test, y_train, y_test = train_test_split(X,y)

In [43]:
print(df.shape)
print("\n")
print("X_train: ", X_train.shape)
print("y_train: ", y_train.shape)
print("X_test: ", X_test.shape)
print("y_test: ", y_test.shape)

(7043, 21)


X_train:  (5282, 45)
y_train:  (5282,)
X_test:  (1761, 45)
y_test:  (1761,)


### SVM

In [44]:
# Fitting classifier to the Training set
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

classifier_svm = SVC()
steps = [
    ('scalar', StandardScaler()),
    ('model', SVC())
]
svm_pipe = Pipeline(steps)

In [45]:
%%time
parameters = { 'model__kernel' : ['linear'],
               'model__C' : [10],
               'model__gamma' : ['scale'],
               'model__random_state' : [42],
               'model__degree' : [1]
}
classifier_svm = GridSearchCV(svm_pipe, parameters, scoring='accuracy', verbose=10, cv=6, n_jobs=-1)
classifier_svm = classifier_svm.fit(X_train, y_train.ravel())

Fitting 6 folds for each of 1 candidates, totalling 6 fits
CPU times: total: 14.8 s
Wall time: 37 s


In [46]:
y_pred_svm_train = classifier_svm.predict(X_train)
accuracy_svm_train = accuracy_score(y_train, y_pred_svm_train)
print("Training set: ", accuracy_svm_train)

y_pred_svm_test = classifier_svm.predict(X_test)
accuracy_svm_test = accuracy_score(y_test, y_pred_svm_test)
print("Test set: ", accuracy_svm_test)

Training set:  0.80367285119273
Test set:  0.7921635434412265


In [47]:
filename = 'data/svm_model.sav'
pickle.dump(classifier_svm, open(filename, 'wb'))


### Logistic Regression

In [48]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
result = model.fit(X_train, y_train)
prediction_test = model.predict(X_test)
accuracy_LR_test = accuracy_score(y_test, prediction_test)
print("Test set: ", accuracy_LR_test)

Test set:  0.797274275979557


C:\python3.8\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


### Random forest

In [49]:
from sklearn.ensemble import RandomForestClassifier
model_rf = RandomForestClassifier(n_estimators=1000 , oob_score = True, n_jobs = -1,
                                  random_state =50, max_features = "auto",
                                  max_leaf_nodes = 30)
model_rf.fit(X_train, y_train)

# Make predictions
prediction_test = model_rf.predict(X_test)
print (metrics.accuracy_score(y_test, prediction_test))

0.7847813742191937


### ADABoost

In [50]:
from sklearn.ensemble import AdaBoostClassifier
model = AdaBoostClassifier()
model.fit(X_train,y_train)
preds = model.predict(X_test)
metrics.accuracy_score(y_test, preds)

0.7950028392958546

### XGBoost

In [51]:
import xgboost as xgb

steps = [
    ('scalar', StandardScaler()),
    ('model', xgb.XGBClassifier())
]
xgb_pipe = Pipeline(steps)

In [52]:
%%time
parameters = { 'model__min_child_weight': [10],
        'model__gamma': [5],
        'model__subsample': [0.6],
        'model__colsample_bytree': [0.6],
        'model__max_depth': [3],
        'model__random_state' : [42]
}
classifier_xgb = GridSearchCV(xgb_pipe, parameters, scoring='accuracy', verbose=10, cv=6, n_jobs=-1)
classifier_xgb = classifier_xgb.fit(X_train, y_train.ravel())

Fitting 6 folds for each of 1 candidates, totalling 6 fits


C:\python3.8\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[00:06:51] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
CPU times: total: 4.5 s
Wall time: 14.3 s


In [53]:
y_pred_xgb_train = classifier_xgb.predict(X_train)
accuracy_xgb_train = accuracy_score(y_train, y_pred_xgb_train)
print("Training set: ", accuracy_xgb_train)

y_pred_xgb_test = classifier_xgb.predict(X_test)
accuracy_xgb_test = accuracy_score(y_test, y_pred_xgb_test)
print("Test set: ", accuracy_xgb_test)

Training set:  0.8252555850056796
Test set:  0.7984099943214082


In [54]:
filename = 'data/xgb_model.sav'
pickle.dump(classifier_xgb, open(filename, 'wb'))

### Artificial Neural Network

In [57]:
import tensorflow as tf
def leaky_relu(x):
    return tf.nn.leaky_relu(x, alpha=0.01)
ann=tf.keras.models.Sequential()
ann.add(tf.keras.layers.Dense(units=6,activation='relu'))
ann.add(tf.keras.layers.Dense(units=4,activation=leaky_relu))
ann.add(tf.keras.layers.Dense(units=1,activation=leaky_relu))   
ann.compile(optimizer='Adam',loss='binary_crossentropy',metrics=['accuracy'])
ann.fit(X_train,y_train,batch_size=16,epochs=50)

Epoch 1/50
331/331 [==============================] - 2s 3ms/step - loss: 10.8425 - accuracy: 0.2653
Epoch 2/50
331/331 [==============================] - 1s 2ms/step - loss: 10.6499 - accuracy: 0.2707A: 0s - loss: 10.6464 - accuracy: 0.270
Epoch 3/50
331/331 [==============================] - 1s 2ms/step - loss: 10.6926 - accuracy: 0.2673
Epoch 4/50
331/331 [==============================] - 1s 2ms/step - loss: 10.6021 - accuracy: 0.2643
Epoch 5/50
331/331 [==============================] - 1s 3ms/step - loss: 10.4655 - accuracy: 0.2630
Epoch 6/50
331/331 [==============================] - 1s 2ms/step - loss: 7.9658 - accuracy: 0.3514
Epoch 7/50
331/331 [==============================] - 1s 3ms/step - loss: 0.5998 - accuracy: 0.7324
Epoch 8/50
331/331 [==============================] - 1s 2ms/step - loss: 0.5682 - accuracy: 0.7377
Epoch 9/50
331/331 [==============================] - 1s 2ms/step - loss: 0.5654 - accuracy: 0.7371
Epoch 10/50
331/331 [==============================] - 1

In [58]:
from sklearn.metrics import classification_report
y_pred=ann.predict(X_test)
y_pred=(y_pred>0.5)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.82      0.91      0.86      1273
           1       0.67      0.49      0.56       488

    accuracy                           0.79      1761
   macro avg       0.75      0.70      0.71      1761
weighted avg       0.78      0.79      0.78      1761

